In [1]:
from src import SharePricePrediction, PredictValues

In [ ]:
def getFundamentals(c_name, future_years):
    predicted_values = PredictValues(c_name=c_name)
    fundamental_values = predicted_values.getFutureValues(future_year=future_years)
    print(fundamental_values)
    return (fundamental_values)

In [ ]:
fundametal_values = getFundamentals('ITC', 5)

In [ ]:
for val in fundametal_values:
    print(f'{val} : {fundametal_values[val]}')

In [ ]:
test_rev = fundametal_values['future_revenue']

test_rev_arr = [round(float(val), 2) for val in test_rev]
print(test_rev_arr)

In [2]:
import json  # Import JSON module to handle JSON strings

def getSharePrice(c_name, future_years):
    share_price_prediction = SharePricePrediction(company_name=c_name)
    share_price = share_price_prediction.SharePrice(future_years=future_years)
    previous_share_price = share_price['previous_share_price']
    future_share_price = share_price['future_share_price']
    
    # Parse JSON strings to Python dictionaries
    data = {
        'previous_share_price': json.loads(previous_share_price.to_json(orient='records', date_format='iso')),
        'future_share_price': json.loads(future_share_price.to_json(orient='records', date_format='iso'))
    }
    return data

In [3]:
share_price_data = getSharePrice('ITC', 5)

77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117

In [4]:
val = share_price_data['previous_share_price']

In [6]:
print(val[0])

{'Date': '2015-04-10T00:00:00.000', 'Close': 146.8356323242, 'ema200': 146.8356323242}


In [ ]:
from flask import json
import mysql
import mysql.connector
import os
from dotenv import load_dotenv

load_dotenv()
DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PORT = os.getenv('DB_PORT')
DB_PASSWORD = os.getenv('DB_PASSWORD')

class CompaniesDB:
    def __init__(self):
        self.connection = self.dbConnection()

    def dbConnection(self):
        try:
            db_connection = mysql.connector.connect(
                host = DB_HOST,
                user = DB_USER,
                password = DB_PASSWORD,
                database = DB_NAME,
                port = DB_PORT
            )
            status = 200
            return {'connection':db_connection, 'status':status}
        except mysql.connector.Error as err:
            status = 404
            return {'status':status, 'data':f'Database connection error: {err}'}

    def _execute_query(self, query, params=None):
        if self.connection['status'] == 200:
            connection = self.connection['connection']
            cursor = connection.cursor(dictionary=True)
            try:
                cursor.execute(query, params)
                connection.commit()
                return {'status': 200, 'data': cursor.fetchall()}
            except mysql.connector.Error as err:
                connection.rollback()
                return {'status': 500, 'data': f"Error executing query: {err}"}
            finally:
                cursor.close()
        else:
            return {"status": 404, "data": 'Database connection error'}

    def searchName(self, c_name):
        query = 'SELECT * FROM companies WHERE c_name = %s'
        result = self._execute_query(query, (c_name,))
        if result['status'] == 200:
            if not result['data']:
                return {'status': 404, 'data': 'company not found'}
            return {'status': 200, 'data': result['data']}
        return result

    def searchID(self, c_id):
        query = 'SELECT * FROM companies WHERE c_id = %s'
        result = self._execute_query(query, (c_id,))
        if result['status'] == 200:
            if not result['data']:
                return {'status': 404, 'data': 'company not found'}
            return {'status': 200, 'data': result['data']}
        return result

    def get_name_suggestions(self, query):
        if self.connection['status'] == 200:
            connection = self.connection['connection']
            q = 'SELECT c_name FROM companies WHERE c_name LIKE %s LIMIT 5'  # Limit to a reasonable number of suggestions
            cursor = connection.cursor(dictionary=True)
            cursor.execute(q, (f'%{query}%',))
            data = cursor.fetchall()
            suggestions = [item['c_name'] for item in data]
            return suggestions
        else:
            return []

In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS
from companies_db import CompaniesDB  # Assuming your database interface is in companies_db.py

app = Flask(__name__)
CORS(app)

@app.route('/<c_name>/getCompany')
def getCompany(c_name):
    c_db = CompaniesDB()
    c_details = c_db.searchName(c_name)
    print(c_details)
    return jsonify(c_details)

@app.route('/suggestions')
def get_suggestions():
    query = request.args.get('query')
    if query:
        c_db = CompaniesDB()
        suggestions = c_db.get_name_suggestions(query)
        return jsonify({'status': 200, 'suggestions': suggestions})
    return jsonify({'status': 400, 'error': 'Missing query parameter'})

if __name__ == '__main__':
    app.run(debug=True)